In [ ]:
print("hi2")

In [7]:
import csv

def make_person_id(name):
    return name.lower().replace(' ','_').replace('.','').replace('(','').replace(')','').replace('-','') \
    .replace('"','').replace("'",'').replace('?','').replace('\\','').replace('’','').replace('“','') \
    .replace('”','')

with open('/Users/chadarmstrong/Library/Mobile Documents/com~apple~CloudDocs/Archive/Programming/Node/geneology_github/data/Geneology.csv') as csv_file:
    people = list(csv.DictReader(csv_file))

people[100:102]

[{'Name': 'Joy Ann Church',
  'Gender': 'Female',
  'Born': '6/16/1971',
  'Born Location': 'Berwick, PA',
  'Died': '',
  'Died Location': '',
  'Mother': '',
  'Father': '',
  'Married to': 'Randall Lee Murphy',
  'Married Date': '12/28/1996',
  'Married Location': 'Northumberland, PA',
  'Second Marriage to': '',
  'Second Marriage Date': '',
  'Second Marriage Location': '',
  'Third Marriage to': '',
  'Third Marriage Date': '',
  'Notes': 'Divorced on 10/4/2011',
  '': ''},
 {'Name': 'Hannah Joy Murphy',
  'Gender': 'Female',
  'Born': '9/16/2000',
  'Born Location': 'Lewisburg, PA',
  'Died': '',
  'Died Location': '',
  'Mother': 'Joy Ann Church',
  'Father': 'Randall Lee Murphy',
  'Married to': '',
  'Married Date': '',
  'Married Location': '',
  'Second Marriage to': '',
  'Second Marriage Date': '',
  'Second Marriage Location': '',
  'Third Marriage to': '',
  'Third Marriage Date': '',
  'Notes': '',
  '': ''}]

In [8]:
count = 34
for person in people:
    print(str(count) + ' ' + person['Name'])
    count = count + 1

34 Joseph Armstrong
35 Ann Waldron
36 Jehiel Armstrong
37 Rebecca Starks
38 Thomas Waldron Armstrong
39 Miles Aldridge Armstrong
40 Alida A. Adams
41 Frank Miles Armstrong
42 Alice M. Haskins
43 Walter Armstrong
44 Claude Armstrong
45 Pearl Armstrong
46 Aubrey Armstrong
47 Guy Armstrong
48 Richard (Dick) Harry Armstrong
49 Eleanor Bullock
50 Noela Griffin
51 Edgar Harry Armstrong
52 Christine Fae (“Tini”) Chaffee
53 Doris Armstrong
54 Melissa A. Armstrong Adams
55 Laura Jane Armstrong
56 Charles Wesley Mosher
57 Colleen Abigail Armstrong
58 Jerry Quick
59 Michael Edward Quick
60 Elizabeth Anne Quick
61 Edgar David Armstrong
62 Carol Marie Murphy
63 Chad David Armstrong
64 Michael David Killen
65 Cynthia Carson
66 Michael Killen
67 Brett Nolan Armstrong
68 Heather Dawn Seymour
69 Brielle Nicole Armstrong
70 Annika Noel Armstrong
71 Sean Patrick Armstrong
72 Renee Elizabeth Glick
73 Renaissance Lenore Armstrong
74 Mason Quade Armstrong
75 Timothy Lee Armstrong
76 Patricia Dyess
77 Timoth

In [ ]:
for person in people:
    print("CREATE (" + make_person_id(person['Name']) + \
    ":Person {name:'" + person['Name'] + "', birth_date:'" + person['Born'] + "', birth_location:'" + \
person['Born Location'] + "', death_date:'" + person['Died'] + "', death_location:'" + person['Died Location'] + "'})")


In [ ]:
print(people[1])

In [ ]:
print('Joseph Armstrong'.lower().replace(' ','_'))

In [ ]:
person = people[3]
print(person)

In [ ]:
def make_person_id(name):
    return name.lower().replace(' ','_').replace('.','').replace('(','').replace(')','').replace('-','') \
    .replace('"','').replace("'",'').replace('?','').replace('\\','')

In [ ]:
last_relationship = None

for person in people[40:90]:
    p = make_person_id(person['Name'])
    m = make_person_id(person['Mother'])
    f = make_person_id(person['Father'])
    s = make_person_id(person['Married to'])

    if person['Mother'] is not '':
        print("MATCH ({}:Person), ({}:Person) WHERE  {}.name = '{}' and {}.name = '{}'".format(m,p,m,person['Mother'],p,person['Name']))
        print("CREATE ({})-[:MOTHER]->({})".format(m,p))
        print("WITH {},{}".format(m,p))
        last_relationship = m
        #print "RETURN m,p"
    if person['Father'] is not '':
        print("MATCH ({}:Person), ({}:Person) WHERE  {}.name = '{}' and {}.name = '{}'".format(f,p,f,person['Father'],p,person['Name']))
        print("CREATE ({})-[:FATHER]->({})".format(f,p))
        print("WITH {},{}".format(f,p))
        last_relationship = f
        #print "RETURN m,p"
    if person['Married to'] is not '':
        print("MATCH ({}:Person), ({}:Person) WHERE  {}.name = '{}' and {}.name = '{}'".format(s,p,s,person['Married to'],p,person['Name']))
        print("CREATE ({})-[:MARRIED]->({})".format(s,p))
        print("WITH {},{}".format(s,p))
        last_relationship = s

print(f"RETURN {last_relationship or ''}")


In [ ]:
len(people)

In [ ]:
people[97]

In [ ]:
from neo4j.v1 import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "aJZ=Ecq4"))

def print_married_to(name):
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for record in tx.run("MATCH (a:Person)-[:MARRIED]->(f) "
                                 "WHERE a.name = {name} "
                                 "RETURN f.name", name=name):
                print(record["f.name"])

print_married_to("Joseph Armstrong")

In [10]:
%pip install neo4j

%pip install neo4j-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 6.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def make_person(person):
    #print(person)
    statement = ("CREATE (" + make_person_id(person['Name']) + ":Person {name:'{name}', birth_date:'{birth_date}', "
                 "birth_location:'{birth_location}', "
                 "death_date:'{death_date}', death_location:'{death_location}'})")

    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(statement, {"person_id": make_person_id(person['Name']),
                                "name": person['Name'],
                                "birth_date": person['Born'],
                                "birth_location": person['Born Location'],
                                "death_date": person['Died'],
                                "death_location": person['Died Location']})
            print('Created ' + person['Name'])

# for person in people[0:10]:
#     make_person(person)

# for person in people:
#     print "CREATE (" + make_person_id(person['Name']) + \
#     ":Person {name:'" + person['Name'] + "', birth_date:'" + person['Born'] + "', birth_location:'" + \
# person['Born Location'] + "', death_date:'" + person['Died'] + "', death_location:'" + person['Died Location'] + "'})"

In [ ]:
def make_relationships(person):
    p = make_person_id(person['Name'])
    m = make_person_id(person['Mother'])
    f = make_person_id(person['Father'])
    s = make_person_id(person['Married to'])

    with driver.session() as session:
        with session.begin_transaction() as tx:
            if person['Mother'] is not '':
                r = m
                tx.run("MATCH ({}:Person), ({}:Person) WHERE  {}.name = '{}' and {}.name = '{}'"
                        " CREATE ({})-[:MOTHER]->({}) WITH {},{}"
                        " RETURN {},{}".format(r,p,r,person['Mother'],p,person['Name'],r,p,r,p,r,p))
                print("{} - :MOTHER -> {}".format(r,p))
            if person['Father'] is not '':
                r = f
                tx.run("MATCH ({}:Person), ({}:Person) WHERE  {}.name = '{}' and {}.name = '{}'"
                        " CREATE ({})-[:FATHER]->({}) WITH {},{}"
                        " RETURN {},{}".format(r,p,r,person['Father'],p,person['Name'],r,p,r,p,r,p))
                print("{} - :FATHER -> {}".format(r,p))
            if person['Married to'] is not '':
                r = s
                tx.run("MATCH ({}:Person), ({}:Person) WHERE  {}.name = '{}' and {}.name = '{}'"
                        " CREATE ({})-[:MARRIED]->({}) WITH {},{}"
                        " RETURN {},{}".format(r,p,r,person['Married to'],p,person['Name'],r,p,r,p,r,p))
                print("{} - :MARRIED -> {}".format(r,p))

# for person in people[0:10]:
#     make_relationships(person)

In [ ]:
import csv
from neo4j import GraphDatabase

localhost = False

if localhost:
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "aJZ=Ecq4"))
else:
    uri = "bolt://52.205.81.5:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "7i=fBzYZ"))

with open('/Users/chada/Desktop/Archive/Programming/Node/geneology/data/Geneology.csv') as csv_file:
    people = list(csv.DictReader(csv_file))

def deleteAll():
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run('MATCH (n) DETACH DELETE n')
            print('Deleted everything!')

def make_person_id(name):
    return name.lower().replace(' ','_').replace('.','').replace('(','').replace(')','').replace('-','') \
    .replace('"','').replace("'",'').replace('?','').replace('\\','').replace('’','').replace('“','') \
    .replace('”','')

def make_person(person):
    #print(person)
    statement = ("CREATE (" + make_person_id(person['Name']) + ":Person {name:{name}, gender:{gender}, birth_date:{birth_date}, "
                 "birth_location:{birth_location}, "
                 "death_date:{death_date}, death_location:{death_location}})")

    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(statement, {"person_id": make_person_id(person['Name']),
                                "name": person['Name'],
                                "gender": person['Gender'],
                                "birth_date": person['Born'],
                                "birth_location": person['Born Location'],
                                "death_date": person['Died'],
                                "death_location": person['Died Location']})
            print('Created ' + person['Name'])


def make_relationships(person):
    p = make_person_id(person['Name'])
    m = make_person_id(person['Mother'])
    f = make_person_id(person['Father'])
    s = make_person_id(person['Married to'])

    with driver.session() as session:
        with session.begin_transaction() as tx:
            if person['Mother'] != '':
                r = m

                query = (
                    "MATCH (r:Person), (p:Person) "
                    "WHERE r.name = $mother_name AND p.name = $person_name "
                    "CREATE (r)-[:MOTHER]->(p) "
                    "RETURN r, p"
                )
                tx.run(query, mother_name=person['Mother'], person_name=person['Name'])
                print(f"{r} - :MOTHER -> {p}")

                if person['Gender'] == 'Male':
                    query = (
                        "MATCH (p:Person), (r:Person) "
                        "WHERE r.name = $mother_name AND p.name = $person_name "
                        "CREATE (p)-[:SON]->(r) "
                        "RETURN p, r"
                    )
                    tx.run(query, mother_name=person['Mother'], person_name=person['Name'])
                    print(f"{p} - :SON -> {r}")
                else:
                    query = (
                        "MATCH (p:Person), (r:Person) "
                        "WHERE r.name = $mother_name AND p.name = $person_name "
                        "CREATE (p)-[:DAUGHTER]->(r) "
                        "RETURN p, r"
                    )
                    tx.run(query, mother_name=person['Mother'], person_name=person['Name'])
                    print(f"{p} - :DAUGHTER -> {r}")

            if person['Father'] != '':
                r = f

                query = (
                    "MATCH (r:Person), (p:Person) "
                    "WHERE r.name = $father_name AND p.name = $person_name "
                    "CREATE (r)-[:FATHER]->(p) "
                    "RETURN r, p"
                )
                tx.run(query, father_name=person['Father'], person_name=person['Name'])
                print(f"{r} - :FATHER -> {p}")

                if person['Gender'] == 'Male':
                    query = (
                        "MATCH (p:Person), (r:Person) "
                        "WHERE r.name = $father_name AND p.name = $person_name "
                        "CREATE (p)-[:SON]->(r) "
                        "RETURN p, r"
                    )
                    tx.run(query, father_name=person['Father'], person_name=person['Name'])
                    print(f"{p} - :SON -> {r}")
                else:
                    query = (
                        "MATCH (p:Person), (r:Person) "
                        "WHERE r.name = $father_name AND p.name = $person_name "
                        "CREATE (p)-[:DAUGHTER]->(r) "
                        "RETURN p, r"
                    )
                    tx.run(query, father_name=person['Father'], person_name=person['Name'])
                    print(f"{p} - :DAUGHTER -> {r}")

            if person['Married to'] != '':
                r = s

                query = (
                    "MATCH (r:Person), (p:Person) "
                    "WHERE r.name = $spouse_name AND p.name = $person_name "
                    "CREATE (r)-[:MARRIED]->(p) "
                    "RETURN r, p"
                )
                tx.run(query, spouse_name=person['Married to'], person_name=person['Name'])
                print(f"{r} - :MARRIED -> {p}")

deleteAll()

#here is a comment
num_rows_to_process = 500

for person in people[0:num_rows_to_process]:
    make_person(person)

for person in people[0:num_rows_to_process]:
    make_relationships(person)

In [ ]:
def deleteAll():
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run('MATCH (n) DETACH DELETE n')
            print('Deleted everything!')

deleteAll()

In [ ]:
def getEverything():
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for record in tx.run('MATCH (n) OPTIONAL MATCH (n)-[r]->(m) RETURN n, r, m'):
                if record['r'] is not None:
                    record_r = record['r'].type
                else:
                    record_r = 'None'
                if record['m'] is not None:
                    record_m = record['m']['name']
                else:
                    record_m = 'None'

                #print(record)
                print('{} is {} to {}'.format(record['n']['name'], record_r, record_m))

getEverything()